# 플라스틱 이미지 Annotation 
#### 고우주 | kubwa 쿱와

## 1. Labelbox(https://labelstud.io) 설치 및 실행
1. 설치: `!pip install -U label-studio`
1. 실행: `label-studio`
1. Plastic file 5장 `COCO *.json` 포맷으로 annotation

## 2. Annotation 완료 디렉토리 구성
> **plastic/** <br>
ㄴ annotations/ <br>
ㄴ images/

## 4. COCO Visualization: BBox

In [1]:
%pip install pycocotools

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pycocotools: filename=pycocotools-2.0.6-cp39-cp39-linux_x86_64.whl size=111186 sha256=14d43a766b6518dbe5677763df66abc6689cd525c0413280f4bb65d63a4d09fb
  Stored in directory: /home/kubwa/.cache/pip/wheels/2f/58/25/e78f1f766e904a9071266661d20d0bc6644df86bcd160aba11
Successfully built pycocotools
Note: you may need to restart the kernel to use updated packages.


In [1]:
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (20,12)

from pycocotools.coco import COCO
import cv2

def coco_json_viz_bbox(img_file_path, annot_path):

    annFile = annot_path
    coco=COCO(annFile)

    catIds = coco.getCatIds();
    imgIds = coco.getImgIds(catIds=catIds );

    img = coco.loadImgs(1)[0]
    annIds = coco.getAnnIds(imgIds=img['id'], catIds=catIds, iscrowd=None)
    anns = coco.loadAnns(annIds)

    image_array = cv2.imread(img_file_path)
    image_array = cv2.cvtColor(image_array, cv2.COLOR_BGR2RGB)
    print(annFile)

    for ann in anns:
        #polygon = ann['segmentation']
        ann_bbox = ann['bbox']
        ann_catogory_id = ann['category_id']
        
        # bounding box를 시각화  
        start_x = ann_bbox[0]
        start_y = ann_bbox[1]
        right = ann_bbox[0] + ann_bbox[2]
        bottom = ann_bbox[1] + ann_bbox[3]
        cv2.rectangle(image_array, (int(start_x), int(start_y)), (int(right), int(bottom)), (125, 255, 51), thickness=3)

        category = coco.loadCats(ann_catogory_id)[0]
        print(category)
        category_name = category['name']
        cv2.putText(image_array, category_name, (int(start_x - 5), int(start_y - 5)), 
                   cv2.FONT_HERSHEY_SIMPLEX, 3, (125, 255, 51), 2)

    plt.figure(figsize=(8, 10))
    plt.imshow(image_array) 
    plt.axis('off')

c:\Users\parkm\anaconda3\envs\py_3_10_4\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\parkm\anaconda3\envs\py_3_10_4\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
c:\Users\parkm\anaconda3\envs\py_3_10_4\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
img_file_path = '../dataset/plastic/images/plastic_test_001.jpg'
annot_path = '../dataset/plastic/annotations/plastic_test_001.json'
coco_json_viz_bbox(img_file_path, annot_path)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
../dataset/plastic/annotations/plastic_test_001.json
{'id': 4, 'name': 'pe'}
{'id': 3, 'name': 'pp'}
{'id': 3, 'name': 'pp'}
{'id': 1, 'name': 'pet'}
{'id': 1, 'name': 'pet'}


## 4. COCO Visualization: Polygon

In [5]:
from pycocotools.coco import COCO
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2

def coco_json_viz_polygon(img_file_path, annot_path):

    annFile = annot_path
    coco=COCO(annFile)

    cats = coco.loadCats(coco.getCatIds())

    if 'supercategory' in cats:
        nms=[cat['name'] for cat in cats]
        nms = set([cat['supercategory'] for cat in cats])
        print('COCO supercategories: \n{}'.format(' '.join(nms)))
        print('COCO categories: \n{}\n'.format(' '.join(nms)))
    else: 
        nms=[cat['name'] for cat in cats]
        print('COCO categories: \n{}\n'.format(' '.join(nms)))

    catIds = coco.getCatIds(catNms=['top']);
    imgIds = coco.getImgIds(catIds=catIds)
    print(cats)

    img = coco.loadImgs(imgIds[0])[0]

    image = cv2.imread(img_file_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    plt.imshow(image)
    cat_ids = coco.getCatIds()
    anns_ids = coco.getAnnIds(imgIds=img['id'], catIds=cat_ids, iscrowd=None)
    anns = coco.loadAnns(anns_ids)
    coco.showAnns(anns)

In [6]:
img_file_path = '../dataset/plastic/images/plastic_test_001.jpg'
annot_path = '../dataset/plastic/annotations/plastic_test_001.json'
coco_json_viz_polygon(img_file_path, annot_path)

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
COCO categories: 
pet ps pp pe

[{'id': 1, 'name': 'pet'}, {'id': 2, 'name': 'ps'}, {'id': 3, 'name': 'pp'}, {'id': 4, 'name': 'pe'}]


## 5. COCO Visualization Mask

In [7]:
def coco_json_viz_mask(img_file_path, annot_path):

    annFile = annot_path
    coco=COCO(annFile)

    cats = coco.loadCats(coco.getCatIds())

    if 'supercategory' in cats:
        nms=[cat['name'] for cat in cats]
        nms = set([cat['supercategory'] for cat in cats])
        print('COCO supercategories: \n{}'.format(' '.join(nms)))
        print('COCO categories: \n{}\n'.format(' '.join(nms)))
    else: 
        nms=[cat['name'] for cat in cats]
        print('COCO categories: \n{}\n'.format(' '.join(nms)))

    catIds = coco.getCatIds(catNms=['top']);
    imgIds = coco.getImgIds(catIds=catIds)
    print(cats)

    img = coco.loadImgs(imgIds[0])[0]

    image = cv2.imread(img_file_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    plt.imshow(image)
    cat_ids = coco.getCatIds()
    anns_ids = coco.getAnnIds(imgIds=img['id'], catIds=cat_ids, iscrowd=None)
    anns = coco.loadAnns(anns_ids)

    mask = coco.annToMask(anns[0])
    for i in range(len(anns)):
        mask += coco.annToMask(anns[i])

    plt.imshow(mask)
    
    return mask

In [8]:
mask = coco_json_viz_mask(img_file_path, annot_path)

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
COCO categories: 
pet ps pp pe

[{'id': 1, 'name': 'pet'}, {'id': 2, 'name': 'ps'}, {'id': 3, 'name': 'pp'}, {'id': 4, 'name': 'pe'}]
